Fuente: https://github.com/raiyan1102006/ChiMerge

In [1]:
##################
# Indicaciones
##################

# 1. Editar la ruta y nombre del archivo en la sección "Cargar Datos"
# 2. Seleccionar la variable numerica en la sección "Definir Variable Numerica"
# 3. Seleccionar la variable categorica en la sección "Definir Variable Categorica"
# 4. Definir el numero de categorias en la sección "Definir las categorias" 
    # 4.1 El codigo está con 3 categorías, si se requieren una cantidad diferente
    # se hace necesario editar las secciones "Unir filas basado en valores minimos de chi cuadrado"
    # "Calcular los valores de chi cuadrado para cada par de filas" y "Calcular tabla de frecuencias"

In [1]:
##################
# Importar librerias
##################

import pandas as pd
import numpy as np
import os
import math

In [7]:
###########
# Cargar Datos. Si es necesario, editar la ruta del dataset y el nombre del mismo
###########

mainpath = "/home/shade/Maestria/entropy_selection/ChiMerge" #Ruta del dataset. En este caso "Mis Documentos"
filename = "data.csv" #Nombre del archivo CSV del dataset
fullpath = os.path.join(mainpath, filename)

data = pd.read_csv(fullpath)
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,target_class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
###########
# Definir la variable numerica
# En este caso es la primera columna (columna 0). Modificar el numero dentro de data.columns
###########

VariableNumerica = data.columns[0]

In [6]:
data.shape

(150, 5)

In [4]:
###########
# Definir la variable categorica
# En este caso es la ultima columna (columna 4). Modificar el numero dentro de data.columns
###########

VariableCategorica = data.columns[4]
Categorias = data[VariableCategorica].unique()
Categorias

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [9]:
###########
# Definir las categorias
# En este caso hay 3 categorías, si se requiere, se agregan más o menos variables 
# pero es necesario editar algunas secciones del código. 
# Secciones definidas en los comentarios y explicadas en las indicaciones
###########

CategoriaUno = Categorias[0]
CategoriaDos = Categorias[1]
CategoriaTres = Categorias[2]

In [10]:
###############################################
# Unir filas basado en valores minimos de chi cuadrado 
###############################################

def merge_rows(df, feature):

    tdf = df[:-1]
    distinct_values = sorted(set(tdf['chi2']), reverse=False)

    #***********************************************************************#
    #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
    #***********************************************************************#
    col_names =  [feature, CategoriaUno, CategoriaDos, CategoriaTres,'chi2']
    
    #Nuevo dataset
    updated_df  = pd.DataFrame(columns = col_names) 
    
    updated_df_index = 0
    for index, row in df.iterrows(): #Itera sobre el antiguo dataset
        if(index == 0):
            updated_df.loc[len(updated_df)] = df.loc[index]
            updated_df_index += 1
        else:
            if(df.loc[index - 1]['chi2'] == distinct_values[0]): #Unir
                
                #***********************************************************************#
                #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
                #***********************************************************************#
                updated_df.loc[updated_df_index - 1][CategoriaUno] += df.loc[index][CategoriaUno]
                updated_df.loc[updated_df_index - 1][CategoriaDos] += df.loc[index][CategoriaDos]
                updated_df.loc[updated_df_index - 1][CategoriaTres] += df.loc[index][CategoriaTres]
            else:
                updated_df.loc[len(updated_df)] = df.loc[index]
                updated_df_index += 1
                
    updated_df['chi2'] = 0. #Limpia los valores antiguos de chi cuadrado

    return updated_df

In [11]:
#####################
# Funcion chi cuadrado
#####################

def calc_chi2(array):
    shape = array.shape
    n = float(array.sum()) #Numero total de entrada
    row={}
    column={}
    
    #Encuentra la suma de filas
    for i in range(shape[0]):
        row[i] = array[i].sum()
    
    #Encuentra la suma de columnas
    for j in range(shape[1]):
        column[j] = array[:, j].sum()

    chi2 = 0
    
    #Usa la formula de chi cuadrado
    for i in range(shape[0]):
        for j in range(shape[1]):
            eij = row[i] * column[j] / n 
            oij = array[i, j]  
            if eij == 0.:
                chi2 += 0. #Nos aseguramos que el valor no moleste
            else: 
                chi2 += math.pow((oij - eij), 2) / float(eij)
  
    return chi2

In [12]:
##################################################################
# Calcular los valores de chi cuadrado para cada par de filas
##################################################################

def update_chi2_column(contingency_table, feature):
    
    for index, row in contingency_table.iterrows():
        #No queremos trabajar solo con la última fila
        if(index != contingency_table.shape[0] - 1): 
            
            #Prepara una matriz con dos filas de datos a la vez
            list1 = []
            list2 = []
            
            #***********************************************************************#
            #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
            #***********************************************************************#
            list1.append(contingency_table.loc[index][CategoriaUno])
            list1.append(contingency_table.loc[index][CategoriaDos])
            list1.append(contingency_table.loc[index][CategoriaTres])
            list2.append(contingency_table.loc[index + 1][CategoriaUno])
            list2.append(contingency_table.loc[index + 1][CategoriaDos])
            list2.append(contingency_table.loc[index + 1][CategoriaTres])
            prep_chi2 = np.array([np.array(list1), np.array(list2)])
            
            #Calcula los valores de chi cuadrado
            c2 = calc_chi2(prep_chi2)
            
            #Actualiza el dataframe
            contingency_table.loc[index]['chi2'] = c2
    return contingency_table


In [13]:
##############################################
# Calcular tabla de frecuencias
##############################################

def create_contingency_table(dataframe, feature):
    distinct_values = sorted(set(dataframe[feature]), reverse = False)
    
    #***********************************************************************#
    #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
    #***********************************************************************#
    col_names =  [feature, CategoriaUno, CategoriaDos, CategoriaTres, 'chi2']
    my_contingency  = pd.DataFrame(columns = col_names)
    
    #Estos son valores de atributos distintos
    for i in range(len(distinct_values)): 
        temp_df = dataframe.loc[dataframe[feature] == distinct_values[i]]
        count_dict = temp_df["target_class"].value_counts().to_dict()

        #***********************************************************************#
        #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
        #***********************************************************************#
        #Inicializa con frecuencias en cero
        CategoriaUno_count = 0
        CategoriaDos_count = 0
        CategoriaTres_count = 0

        #***********************************************************************#
        #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
        #***********************************************************************#
        #Actualiza si es necesario
        if CategoriaUno in count_dict:
            CategoriaUno_count = count_dict[CategoriaUno]
        if CategoriaDos in count_dict:
            CategoriaDos_count = count_dict[CategoriaDos]
        if CategoriaTres in count_dict:
            CategoriaTres_count = count_dict[CategoriaTres]

        #***********************************************************************#
        #******** EDITAR SI SE AGREGARON MAS O MENOS DE TRES CATEGORIAS ********#
        #***********************************************************************#
        new_row = [distinct_values[i], CategoriaUno_count, CategoriaDos_count, CategoriaTres_count, 0]
        my_contingency.loc[len(my_contingency)] = new_row

    return my_contingency

In [14]:
####################
# Funcion Chi Merge
####################

def chimerge(feature, data, max_interval):
    df = data.sort_values(by = [feature], ascending = True).reset_index()
    
    #Genera la tabla de frecuencias
    contingency_table = create_contingency_table(df, feature)

    #Calcula el numero inicial de intervalos
    num_intervals = contingency_table.shape[0] 

    #Mantener el ciclo hasta que se cumpla la condición de intervalo máximo
    while num_intervals > max_interval: 
        #Calcular chi cuadrado para pares de filas adyacentes
        chi2_df = update_chi2_column(contingency_table, feature) 
        
        #Fusionar filas basadas en los valores mínimos de chi cuadrado
        contingency_table = merge_rows(chi2_df, feature)
        
        #Actualizar numero de intervalos
        num_intervals = contingency_table.shape[0]               

    #Imprimir resultados
    print('Los intervalos del atributo numerico ' + feature + ' son:')
    for index, row in contingency_table.iterrows():
        if(index != contingency_table.shape[0] - 1):
            for index2, row2 in df.iterrows():
                if df.loc[index2][feature] < contingency_table.loc[index + 1][feature]:
                    temp = df.loc[index2][feature]
        else:
            temp = df[feature].iloc[-1]
        print("[" + str(contingency_table.loc[index][feature]) + ", " + str(temp) + "]")

In [23]:
for feature in [VariableNumerica]:
    chimerge(feature = feature, data = data, max_interval = 6)

Los intervalos del atributo numerico sepal_length son:
[4.3, 4.8]
[4.9, 4.9]
[5.0, 5.4]
[5.5, 5.7]
[5.8, 7.0]
[7.1, 7.9]
